<a href="https://colab.research.google.com/github/ganeshred/NLPAuthorshipAttributionNLM/blob/main/sentence_level_bi_LSTM_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [67]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [68]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [69]:
import pandas as pd

In [70]:
df = pd.read_csv('/content/gdrive/MyDrive/Task3.csv')

In [71]:
ctrl_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_ctrl.csv')
fair_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_fair.csv')
gpt_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_gpt.csv')
gpt2_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_gpt2.csv')
grover_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_grover.csv')
human_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_human.csv')
pplm_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_pplm.csv')
xlm_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_xlm.csv')
xlnet_df = pd.read_csv('/content/gdrive/MyDrive/NLP/new_xlnet.csv')
instruct_df = pd.read_csv('/content/gdrive/MyDrive/NLP/instruct.csv')
gpt3_df = pd.read_csv('/content/gdrive/MyDrive/NLP/gpt3.csv')

In [72]:
ctrl_df = ctrl_df[['Generation','label']]
fair_df = fair_df[['Generation','label']]
gpt_df = gpt_df[['Generation','label']]
gpt3_df = gpt3_df[['Generation','label']]
gpt2_df = gpt2_df[['Generation','label']]
grover_df = grover_df[['Generation','label']]
human_df = human_df[['Generation','label']]
pplm_df = pplm_df[['Generation','label']]
xlm_df = xlm_df[['Generation','label']]
xlnet_df = xlnet_df[['Generation','label']]
instruct_df = instruct_df[['Generation','label']]


In [73]:
combined_df = pd.concat([ctrl_df,fair_df,gpt_df,gpt3_df,gpt2_df,grover_df,human_df,pplm_df,xlm_df,xlnet_df,instruct_df], ignore_index=True)


In [74]:
combined_df[['Generation']] = combined_df[['Generation']]. astype(str)

In [75]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # convert text to lowercase
    text = text.lower()

    # remove punctuation except fullstops
    exclude = set(string.punctuation) - set('.')
    text = ''.join(ch for ch in text if ch not in exclude)

    # tokenize text into words
    words = word_tokenize(text)

    # remove stopwords
    stop_words = set(stopwords.words("english"))
    filtered_words = [word for word in words if word not in stop_words]

    # lemmatize words
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

    # join words back into a string
    processed_text = " ".join(lemmatized_words)

    return processed_text

# def split_sentences(paragraph):
#     sentences = nltk.sent_tokenize(paragraph)
#     sentences = [preprocess_text(sentence) for sentence in sentences]
#     return sentences

def split_sentences(paragraph):
    sentences = nltk.sent_tokenize(paragraph)
    sentences = [preprocess_text(sentence) for sentence in sentences]

    # merge every other sentence
    merged_sentences = []
    for i in range(0, len(sentences), 5):
        if i+5 < len(sentences):
            merged_sentence = sentences[i] + '. ' + sentences[i+1] + '. ' + sentences[i+2] + '. ' + sentences[i+3] + '. ' + sentences[i+4]
            merged_sentences.append(merged_sentence)
        else:
            merged_sentences.append(sentences[i])

    return merged_sentences


combined_df
# apply the function to the paragraph column and create a new column with the sentences
combined_df['Generation_sentences'] = combined_df['Generation'].apply(split_sentences)
combined_df1 = combined_df.explode('Generation_sentences')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [76]:
len(combined_df)

11726

In [77]:
len(combined_df1)

48730

In [78]:
combined_df1.head()

,Generation,label,Generation_sentences
0,latest headlines on cnn business tl;dr the u.s...,ctrl,latest headline cnn business tldr u.s. governm...
0,latest headlines on cnn business tl;dr the u.s...,ctrl,click close article ... read morehttpwww.reute...
0,latest headlines on cnn business tl;dr the u.s...,ctrl,fun reading answer question posted reddit user...
0,latest headlines on cnn business tl;dr the u.s...,ctrl,long term 5 year old .
1,china wants to take a victory lap over its han...,ctrl,china want take victory lap handling coronavir...


In [79]:
combined_df1

,Generation,label,Generation_sentences
0,latest headlines on cnn business tl;dr the u.s...,ctrl,latest headline cnn business tldr u.s. governm...
0,latest headlines on cnn business tl;dr the u.s...,ctrl,click close article ... read morehttpwww.reute...
0,latest headlines on cnn business tl;dr the u.s...,ctrl,fun reading answer question posted reddit user...
0,latest headlines on cnn business tl;dr the u.s...,ctrl,long term 5 year old .
1,china wants to take a victory lap over its han...,ctrl,china want take victory lap handling coronavir...
...,...,...,...
11721,All of it!,instructgpt,
11722,Space stations must be kept clean to prevent t...,instructgpt,space station must kept clean prevent spread b...
11723,It is not mandatory to pay a year's rent up fr...,instructgpt,mandatory pay year rent front city .
11724,". Stay up to date with the latest news, featur...",instructgpt,.


In [80]:
combined_df1['label'] = combined_df1['label'].str.lower()

In [81]:
combined_df1['label'].unique()

array(['ctrl', 'fair', 'gpt', 'gpt3', 'gpt2', 'grover', 'human', 'pplm',
       'xlm', 'xlnet', 'instructgpt'], dtype=object)

In [82]:
# import nltk
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# import string

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

# def preprocess_text(text):
#     # convert text to lowercase
#     text = text.lower()

#     # remove punctuation except fullstops
#     exclude = set(string.punctuation) - set('.')
#     text = ''.join(ch for ch in text if ch not in exclude)

#     # tokenize text into words
#     words = word_tokenize(text)

#     # remove stopwords
#     stop_words = set(stopwords.words("english"))
#     filtered_words = [word for word in words if word not in stop_words]

#     # lemmatize words
#     lemmatizer = WordNetLemmatizer()
#     lemmatized_words = [lemmatizer.lemmatize(word) for word in filtered_words]

#     # join words back into a string
#     processed_text = " ".join(lemmatized_words)

#     return processed_text

# # def split_sentences(paragraph):
# #     sentences = nltk.sent_tokenize(paragraph)
# #     sentences = [preprocess_text(sentence) for sentence in sentences]
# #     return sentences

# def split_sentences(paragraph):
#     sentences = nltk.sent_tokenize(paragraph)
#     sentences = [preprocess_text(sentence) for sentence in sentences]

#     # merge every other sentence
#     merged_sentences = []
#     for i in range(0, len(sentences), 5):
#         if i+5 < len(sentences):
#             merged_sentence = sentences[i] + '. ' + sentences[i+1] + '. ' + sentences[i+2] + '. ' + sentences[i+3] + '. ' + sentences[i+4]
#             merged_sentences.append(merged_sentence)
#         else:
#             merged_sentences.append(sentences[i])

#     return merged_sentences


# # apply the function to the paragraph column and create a new column with the sentences
# ctrl_df['Generation_sentences'] = ctrl_df['Generation'].apply(split_sentences)
# ctrl_df1 = ctrl_df.explode('Generation_sentences')


In [83]:
# ctrl_df1.head(20)

In [84]:
df = combined_df1[['Generation_sentences','label']]

In [85]:
# df[['Generation']] = df[['Generation']]. astype(str)

In [86]:
df.head()

,Generation_sentences,label
0,latest headline cnn business tldr u.s. governm...,ctrl
0,click close article ... read morehttpwww.reute...,ctrl
0,fun reading answer question posted reddit user...,ctrl
0,long term 5 year old .,ctrl
1,china want take victory lap handling coronavir...,ctrl


In [87]:
len(df)

48730

In [88]:
# text_field = data.Field(
#     sequential=True,
#     tokenize='basic_english', 
#     fix_length=500,
#     lower=True
# )
TEXT = data.Field(tokenize = 'spacy', tokenizer_language='en_core_web_sm' , include_lengths = True)
LABEL = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)

# label_field = data.Field(sequential=False, use_vocab=False)

# sadly have to apply preprocess manually
preprocessed_text = df['Generation_sentences'].apply(
    lambda x: TEXT.preprocess(x)
)

# load fastext simple embedding with 300d
TEXT.build_vocab(
    preprocessed_text, 
    max_size = 30000,
    vectors='glove.6B.50d',
    unk_init = torch.Tensor.normal_
)

# get the vocab instance
vocab = TEXT.vocab

In [89]:
preprocessed_text

0        [latest, headline, cnn, business, tldr, u.s, ....
0        [click, close, article, ..., read, morehttpwww...
0        [fun, reading, answer, question, posted, reddi...
0                            [long, term, 5, year, old, .]
1        [china, want, take, victory, lap, handling, co...
                               ...                        
11721                                                   []
11722    [space, station, must, kept, clean, prevent, s...
11723         [mandatory, pay, year, rent, front, city, .]
11724                                                  [.]
11725    [order, strengthen, trust, transparency, onlin...
Name: Generation_sentences, Length: 48730, dtype: object

In [90]:
ltoi = {l: i for i, l in enumerate(df['label'].unique())}

In [91]:
df['label1'] = df['label'].apply(lambda y: ltoi[y])

class DataFrameDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                data.Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

In [92]:
df1=df[['Generation_sentences','label1']]

In [93]:
examples=[]
for i,row in df1.iterrows():
  # print(example)
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

In [94]:
# examples[2000].label
c=0
for i in examples:
  if type(i.label) != int:
    c+=1

In [95]:
ltoi

{'ctrl': 0,
 'fair': 1,
 'gpt': 2,
 'gpt3': 3,
 'gpt2': 4,
 'grover': 5,
 'human': 6,
 'pplm': 7,
 'xlm': 8,
 'xlnet': 9,
 'instructgpt': 10}

In [96]:
dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])

In [97]:
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.65, 0.15, 0.2], stratified=True, strata_field='label')

In [98]:
len(train_dataset)

31675

In [99]:
train_dataset[0].text

['information',
 'visit',
 'www.vietnam.traveltourism',
 '..',
 'cnn.com',
 'travel',
 'writer',
 'michael',
 'j.',
 'de',
 'la',
 'merced',
 'contributed',
 'reporting',
 'cnn',
 'staff',
 'writer',
 'andy',
 'kroll',
 'john',
 'murgatroyd',
 'jim',
 'bittermann',
 'writing',
 'carol',
 'landy',
 'new',
 'york',
 'editing',
 'eric',
 'walsh',
 'peter',
 'cooneyhttpwww.nytimes.com20110203world03china.htmlr1hp',
 'tldr',
 'new',
 'law',
 'would',
 'allow',
 'chinese',
 'citizen',
 'ethnic',
 'minority',
 'foreigner',
 'living',
 'legally',
 'since',
 '1997',
 '—',
 'whose',
 'child',
 'born',
 '1978',
 '—',
 'citizenship',
 'right',
 'equal',
 'enjoyed',
 'existing',
 'law',
 '..',
 'could',
 'year',
 'become',
 'citizen',
 '..',
 'earlier',
 'version',
 'misstated',
 'age',
 '..',
 'must',
 'turn',
 '21',
 'within',
 'three',
 'year',
 'time',
 'legal',
 'resident',
 'automatically',
 'acquire',
 'citizenship',
 '.']

In [100]:
ltoi

{'ctrl': 0,
 'fair': 1,
 'gpt': 2,
 'gpt3': 3,
 'gpt2': 4,
 'grover': 5,
 'human': 6,
 'pplm': 7,
 'xlm': 8,
 'xlnet': 9,
 'instructgpt': 10}

In [101]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)
# device = 'cpu'
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cpu


In [102]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim, padding_idx = pad_idx)
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = n_layers, dropout = dropout, bidirectional = bidirectional)
        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim * n_layers, output_dim)
        #4. Initialize Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        embedding = self.embedding(text)
        embedding_dropout = self.dropout(embedding)
        #2. Run the LSTM along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions]; #hidden = [num layers * num directions, batch size, hid dim]
        output, (hidden,cell) = self.lstm(embedding_dropout)
        #3. Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        concat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(concat)     
        return self.fc(hidden)

In [103]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 126
OUTPUT_DIM = 11
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [104]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,065,347 trainable parameters


In [105]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([30002, 50])


In [106]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.7372,  0.3910,  0.5158],
        [ 0.1825, -0.0737,  0.3147,  ...,  1.3226, -2.6796, -0.1297],
        [ 1.2907,  0.2612, -0.5862,  ..., -0.7711, -1.8497,  0.2240],
        ...,
        [-0.0997, -0.7484, -0.3954,  ..., -0.0219, -0.4788, -1.1504],
        [-0.6693, -0.0655, -0.2100,  ...,  0.0623, -0.5882, -0.1216],
        [-0.4310, -0.7707,  0.1562,  ..., -0.5575,  0.9139,  0.3053]])

In [107]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 1.2907,  0.2612, -0.5862,  ..., -0.7711, -1.8497,  0.2240],
        ...,
        [-0.0997, -0.7484, -0.3954,  ..., -0.0219, -0.4788, -1.1504],
        [-0.6693, -0.0655, -0.2100,  ...,  0.0623, -0.5882, -0.1216],
        [-0.4310, -0.7707,  0.1562,  ..., -0.5575,  0.9139,  0.3053]])


In [108]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [109]:
criterion = nn.CrossEntropyLoss()

# device='cpu'
model = model.to(device)
criterion = criterion.to(device)

In [110]:
def multiclass_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    # get the index of the highest prediction for each instance
    _, preds = torch.max(preds, dim=1)
    correct = (preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [111]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # print(batch.text)
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        target = batch.label
        target = target.type(torch.LongTensor)

        # print(target)

        # print(predictions)
        
        loss = criterion(predictions.to(device), target.to(device))
        
        acc = multiclass_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [112]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = multiclass_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()



    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [113]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [118]:
N_EPOCHS = 15
path = F"/content/gdrive/My Drive/sentence_bilstm_model_task3.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 3m 17s
	Train Loss: 0.742 | Train Acc: 73.13%
	 Val. Loss: 0.982 |  Val. Acc: 65.42%
Epoch: 02 | Epoch Time: 3m 17s
	Train Loss: 0.708 | Train Acc: 74.45%
	 Val. Loss: 0.962 |  Val. Acc: 66.23%
Epoch: 03 | Epoch Time: 3m 23s
	Train Loss: 0.671 | Train Acc: 75.73%
	 Val. Loss: 0.985 |  Val. Acc: 66.69%
Epoch: 04 | Epoch Time: 3m 20s
	Train Loss: 0.642 | Train Acc: 76.95%
	 Val. Loss: 0.929 |  Val. Acc: 68.26%
Epoch: 05 | Epoch Time: 3m 24s
	Train Loss: 0.608 | Train Acc: 78.21%
	 Val. Loss: 1.006 |  Val. Acc: 67.62%
Epoch: 06 | Epoch Time: 3m 17s
	Train Loss: 0.581 | Train Acc: 79.07%
	 Val. Loss: 0.955 |  Val. Acc: 68.82%
Epoch: 07 | Epoch Time: 3m 16s
	Train Loss: 0.561 | Train Acc: 80.06%
	 Val. Loss: 1.018 |  Val. Acc: 66.29%
Epoch: 08 | Epoch Time: 3m 17s
	Train Loss: 0.543 | Train Acc: 80.66%
	 Val. Loss: 0.973 |  Val. Acc: 68.38%
Epoch: 09 | Epoch Time: 3m 17s
	Train Loss: 0.516 | Train Acc: 81.79%
	 Val. Loss: 1.022 |  Val. Acc: 68.08%
Epoch: 10 | Epoch T

In [119]:
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.025 | Test Acc: 70.49%
